<a href="https://colab.research.google.com/github/Sopralapanca/CommonLitChallenge/blob/main/PrepareDataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [14]:
!pip install pyspellchecker

In [15]:
!git clone "https://github.com/Sopralapanca/CommonLitChallenge.git"

fatal: destination path 'CommonLitChallenge' already exists and is not an empty directory.


In [16]:
import pandas as pd
summaries_train_path = "/content/CommonLitChallenge/data/summaries_train.csv"
prompt_train_path = "/content/CommonLitChallenge/data/prompts_train.csv"

summaries_test_path = "/content/CommonLitChallenge/data/summaries_test.csv"
prompt_test_path = "/content/CommonLitChallenge/data/prompts_test.csv"

train_data = pd.read_csv(summaries_train_path, sep=',', index_col=0)
prompt_data = pd.read_csv(prompt_train_path, sep=',', index_col=0)


In [17]:
from nltk.util import ngrams
from collections import Counter
import nltk
nltk.download('punkt')  # Download the required resources for tokenization

def count_ngrams(text, n):
  words = nltk.word_tokenize(text)
  ngram_counts = Counter(ngrams(words, n))
  return ngram_counts

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [18]:
del_columns = []

for n in range(2, 5):
  col = f"{n}grams-prompttext-count"
  prompt_data[col] = prompt_data.apply(lambda row: count_ngrams(row["prompt_text"], n), axis=1)

  del_columns.append(col)

  col = f"{n}grams-text-count"
  train_data[col] = train_data.apply(lambda row: count_ngrams(row["text"], n), axis=1)

  del_columns.append(col)

In [19]:
training_data = train_data.merge(prompt_data, on='prompt_id')

In [24]:
def count_cooccurring_ngrams(text, prompt_text):
    cooccurring_count = sum((text & prompt_text).values())
    return cooccurring_count

In [25]:
for n in range(2,5):
  text_col = f"{n}grams-text-count"
  prompt_col = f"{n}grams-prompttext-count"

  new_col=f"{n}grams-cooccurence-count"
  training_data[new_col] = training_data.apply(lambda row: count_cooccurring_ngrams(row[text_col], row[prompt_col]), axis=1)



In [26]:
training_data.head()

,prompt_id,text,content,wording,2grams-text-count,3grams-text-count,4grams-text-count,prompt_question,prompt_title,prompt_text,2grams-prompttext-count,3grams-prompttext-count,4grams-prompttext-count,2grams-cooccurence-count,3grams-cooccurence-count,4grams-cooccurence-count
0,814d6b,The third wave was an experimentto see how peo...,0.205683,0.380538,"{('The', 'third'): 1, ('third', 'wave'): 1, ('...","{('The', 'third', 'wave'): 1, ('third', 'wave'...","{('The', 'third', 'wave', 'was'): 1, ('third',...",Summarize how the Third Wave developed over su...,The Third Wave,Background \nThe Third Wave experiment took pl...,"{('Background', 'The'): 1, ('The', 'Third'): 2...","{('Background', 'The', 'Third'): 1, ('The', 'T...","{('Background', 'The', 'Third', 'Wave'): 1, ('...",4,0,0
1,814d6b,The Third Wave developed rapidly because the ...,3.272894,3.219757,"{('The', 'Third'): 2, ('Third', 'Wave'): 4, ('...","{('The', 'Third', 'Wave'): 2, ('Third', 'Wave'...","{('The', 'Third', 'Wave', 'developed'): 1, ('T...",Summarize how the Third Wave developed over su...,The Third Wave,Background \nThe Third Wave experiment took pl...,"{('Background', 'The'): 1, ('The', 'Third'): 2...","{('Background', 'The', 'Third'): 1, ('The', 'T...","{('Background', 'The', 'Third', 'Wave'): 1, ('...",35,6,0
2,814d6b,The third wave only started as an experiment w...,0.205683,0.380538,"{('The', 'third'): 1, ('third', 'wave'): 1, ('...","{('The', 'third', 'wave'): 1, ('third', 'wave'...","{('The', 'third', 'wave', 'only'): 1, ('third'...",Summarize how the Third Wave developed over su...,The Third Wave,Background \nThe Third Wave experiment took pl...,"{('Background', 'The'): 1, ('The', 'Third'): 2...","{('Background', 'The', 'Third'): 1, ('The', 'T...","{('Background', 'The', 'Third', 'Wave'): 1, ('...",16,8,5
3,814d6b,The experimen was orginally about how even whe...,0.567975,0.969062,"{('The', 'experimen'): 1, ('experimen', 'was')...","{('The', 'experimen', 'was'): 1, ('experimen',...","{('The', 'experimen', 'was', 'orginally'): 1, ...",Summarize how the Third Wave developed over su...,The Third Wave,Background \nThe Third Wave experiment took pl...,"{('Background', 'The'): 1, ('The', 'Third'): 2...","{('Background', 'The', 'Third'): 1, ('The', 'T...","{('Background', 'The', 'Third', 'Wave'): 1, ('...",28,10,4
4,814d6b,The third wave developed so quickly due to the...,-0.910596,-0.081769,"{('The', 'third'): 1, ('third', 'wave'): 1, ('...","{('The', 'third', 'wave'): 1, ('third', 'wave'...","{('The', 'third', 'wave', 'developed'): 1, ('t...",Summarize how the Third Wave developed over su...,The Third Wave,Background \nThe Third Wave experiment took pl...,"{('Background', 'The'): 1, ('The', 'Third'): 2...","{('Background', 'The', 'Third'): 1, ('The', 'T...","{('Background', 'The', 'Third', 'Wave'): 1, ('...",5,0,0


In [27]:
training_data.drop(columns=del_columns, inplace=True)

In [28]:
training_data["text_length"] = training_data["text"].apply(len)
training_data["prompt_text_length"] = training_data["prompt_text"].apply(len)
training_data["length_ratio"] = training_data["text_length"] / training_data["prompt_text_length"]

In [29]:
from spellchecker import SpellChecker

spell = SpellChecker()

def misspelled_counter(text):
  words = text.split()
  misspelled = spell.unknown(words)
  return len(misspelled)

In [31]:
training_data["misspelled_counter"] = training_data["text"].apply(lambda x: misspelled_counter(x))

In [32]:
# normalize the data taking into consideration the prompt title

from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()

def normalize_col(training_data, col):
  # Create a new DataFrame to store the normalized text length values
  normalized_df = pd.DataFrame()

  # Group by 'prompt_title' and apply the normalization separately for each group
  for title, group in training_data.groupby('prompt_title'):
      normalized_text_length = scaler.fit_transform(group[col].values.reshape(-1, 1))
      new_name = "normalized_"+col
      group[new_name] = normalized_text_length
      normalized_df = pd.concat([normalized_df, group])
  training_data = normalized_df.copy()
  return training_data

In [33]:
training_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7165 entries, 0 to 7164
Data columns (total 14 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   prompt_id                 7165 non-null   object 
 1   text                      7165 non-null   object 
 2   content                   7165 non-null   float64
 3   wording                   7165 non-null   float64
 4   prompt_question           7165 non-null   object 
 5   prompt_title              7165 non-null   object 
 6   prompt_text               7165 non-null   object 
 7   2grams-cooccurence-count  7165 non-null   int64  
 8   3grams-cooccurence-count  7165 non-null   int64  
 9   4grams-cooccurence-count  7165 non-null   int64  
 10  text_length               7165 non-null   int64  
 11  prompt_text_length        7165 non-null   int64  
 12  length_ratio              7165 non-null   float64
 13  misspelled_counter        7165 non-null   int64  
dtypes: float

In [35]:
normalize_cols = ["text_length", "prompt_text_length", "misspelled_counter", "2grams-cooccurence-count", "3grams-cooccurence-count", "4grams-cooccurence-count"]
for col in normalize_cols:
  training_data = normalize_col(training_data, col)

In [36]:
training_data.head()

,prompt_id,text,content,wording,prompt_question,prompt_title,prompt_text,2grams-cooccurence-count,3grams-cooccurence-count,4grams-cooccurence-count,text_length,prompt_text_length,length_ratio,misspelled_counter,normalized_text_length,normalized_prompt_text_length,normalized_misspelled_counter,normalized_2grams-cooccurence-count,normalized_3grams-cooccurence-count,normalized_4grams-cooccurence-count
3099,3b9047,"In Egypt, there were many occupations and soci...",3.128928,4.231226,"In complete sentences, summarize the structure...",Egyptian Social Structure,Egyptian society was structured like a pyramid...,57,23,11,1370,3329,0.411535,32,0.328280,0.0,0.316832,0.143216,0.058376,0.028205
3100,3b9047,The highest class was Pharaohs these people we...,-0.210614,-0.471415,"In complete sentences, summarize the structure...",Egyptian Social Structure,Egyptian society was structured like a pyramid...,6,5,4,157,3329,0.047161,5,0.011239,0.0,0.049505,0.015075,0.012690,0.010256
3101,3b9047,The Egyptian society is really different from ...,0.205683,0.380538,"In complete sentences, summarize the structure...",Egyptian Social Structure,Egyptian society was structured like a pyramid...,8,0,0,453,3329,0.136077,7,0.088604,0.0,0.069307,0.020101,0.000000,0.000000
3102,3b9047,We have the gods and then Logan and If Logan ...,-1.547163,-1.461245,"In complete sentences, summarize the structure...",Egyptian Social Structure,Egyptian society was structured like a pyramid...,1,0,0,131,3329,0.039351,3,0.004443,0.0,0.029703,0.002513,0.000000,0.000000
3103,3b9047,The social classes are different because they ...,-0.066112,-0.715083,"In complete sentences, summarize the structure...",Egyptian Social Structure,Egyptian society was structured like a pyramid...,41,35,34,437,3329,0.131271,6,0.084422,0.0,0.059406,0.103015,0.088832,0.087179


In [37]:
training_data.to_csv('dataset.csv', index=False)

In [ ]:
import spacy
import torch
import string
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
import re


def preprocessText(text, intense=False):
    try:
        # replace newline with space
        text = text.replace("\n", "")

        text = text.replace('\r', '')
        # Replace curly apostrophe with straight single quote
        text = text.replace('’', "'")

        # Normalize spaces around punctuation marks
        text = re.sub(r'\s+', ' ', text)
        text = re.sub(r'\s([.,!?])', r'\1', text)
        text = re.sub(r'([.,!?])\s', r'\1', text)
        text = text.strip()

        if intense:
            # lower case
            text = text.lower()

            # remove punctuations
            translator = str.maketrans("", "", string.punctuation)
            text = text.translate(translator)

            # split text
            words = text.split()

            # stop word removal
            stop_words = spacy.lang.en.stop_words.STOP_WORDS
            words = [w for w in words if not w in stop_words]

            # stemming
            stemmer = PorterStemmer()
            words = [stemmer.stem(w) for w in words]

            # lemmatization
            lemmatizer = WordNetLemmatizer()
            words = [lemmatizer.lemmatize(w) for w in words]

            # return pre-processed paragraph text
            text = ' '.join(words)

        return text
    except:
        return text

In [ ]:
training_data.corrwith(training_data["content"])

<ipython-input-35-3ecd97e2210c>:1: FutureWarning: The default value of numeric_only in DataFrame.corrwith is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.
  training_data.corrwith(training_data["content"])


content                     1.000000
wording                     0.751380
text_length                 0.797244
prompt_text_length         -0.038532
misspelled_counter          0.694396
cooccurring_count           0.498155
2grams-cooccurring-count    0.498155
3grams-cooccurring-count    0.363915
4grams-cooccurring-count    0.319285
dtype: float64

In [ ]:
training_data.corrwith(training_data["wording"])

<ipython-input-37-360ae61d465e>:1: FutureWarning: The default value of numeric_only in DataFrame.corrwith is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.
  training_data.corrwith(training_data["wording"])


content                     0.751380
wording                     1.000000
text_length                 0.540138
prompt_text_length         -0.121241
misspelled_counter          0.412939
cooccurring_count           0.133163
2grams-cooccurring-count    0.133163
3grams-cooccurring-count    0.012620
4grams-cooccurring-count   -0.016339
dtype: float64

In [ ]:
training_data.head()

,prompt_id,text,content,wording,prompt_question,prompt_title,prompt_text,text_length,prompt_text_length,misspelled_counter,...,2grams-cooccurring-count,3grams-cooccurring-count,4grams-cooccurring-count,normalized_text_length,normalized_prompt_text_length,normalized_misspelled_counter,normalized_2grams-cooccurring-count,normalized_3grams-cooccurring-count,normalized_4grams-cooccurring-count,normalized_normalized_text_length
3099,3b9047,"In Egypt, there were many occupations and soci...",3.128928,4.231226,"In complete sentences, summarize the structure...",Egyptian Social Structure,Egyptian society was structured like a pyramid...,1370,3329,32,...,57,23,11,0.328280,0.0,0.316832,0.143216,0.058376,0.028205,0.328280
3100,3b9047,The highest class was Pharaohs these people we...,-0.210614,-0.471415,"In complete sentences, summarize the structure...",Egyptian Social Structure,Egyptian society was structured like a pyramid...,157,3329,5,...,6,5,4,0.011239,0.0,0.049505,0.015075,0.012690,0.010256,0.011239
3101,3b9047,The Egyptian society is really different from ...,0.205683,0.380538,"In complete sentences, summarize the structure...",Egyptian Social Structure,Egyptian society was structured like a pyramid...,453,3329,7,...,8,0,0,0.088604,0.0,0.069307,0.020101,0.000000,0.000000,0.088604
3102,3b9047,We have the gods and then Logan and If Logan ...,-1.547163,-1.461245,"In complete sentences, summarize the structure...",Egyptian Social Structure,Egyptian society was structured like a pyramid...,131,3329,3,...,1,0,0,0.004443,0.0,0.029703,0.002513,0.000000,0.000000,0.004443
3103,3b9047,The social classes are different because they ...,-0.066112,-0.715083,"In complete sentences, summarize the structure...",Egyptian Social Structure,Egyptian society was structured like a pyramid...,437,3329,6,...,41,35,34,0.084422,0.0,0.059406,0.103015,0.088832,0.087179,0.084422
